# Understanding the `Exp` Operation

The `Exp` operation calls the internal [`SpreadZ`](https://github.com/microsoft/qsharp-runtime/blob/70c881d8b93f290bca6145fb5fda28e5dbc57827/src/Simulation/TargetDefinitions/Decompositions/Utils.qs#L7-L16) operation:

In [3]:
operation SpreadZ(from : Qubit, to : Qubit[]) : Unit is Adj {
    if (Length(to) > 0) {
        CNOT(to[0], from);
        if (Length(to) > 1) {
            let half = Length(to) / 2;
            SpreadZ(to[0], to[half + 1 .. Length(to) - 1]);
            SpreadZ(from, to[1 .. half]);
        }
    }
}


operation ApplySpreadZ(n : Int) : Unit {
    use (fromQubit, toReg) = (Qubit(), Qubit[n]);
    SpreadZ(fromQubit, toReg);
}

SpreadZ, ApplySpreadZ

In [59]:
%trace ApplySpreadZ n=3

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

https://github.com/microsoft/qsharp-runtime/blob/2b82379d284a194a7b613641caca0e550c2db30d/src/Simulation/TargetDefinitions/Decompositions/Utils.qs#L18-L43

In [10]:
operation MapPauli (qubit : Qubit, from : Pauli, to : Pauli) : Unit is Adj {
    if (from == to) {
    }
    elif ((from == PauliZ and to == PauliX) or (from == PauliX and to == PauliZ)) {
        H(qubit);
    }
    elif (from == PauliZ and to == PauliY) {
        H(qubit);
        S(qubit);
        H(qubit);
    }
    elif (from == PauliY and to == PauliZ) {
        H(qubit);
        Adjoint S(qubit);
        H(qubit);
    }
    elif (from == PauliY and to == PauliX) {
        S(qubit);
    }
    elif (from == PauliX and to == PauliY) {
        Adjoint S(qubit);
    }
    else {
        fail "Unsupported input";
    }
}

MapPauli

https://github.com/microsoft/qsharp-runtime/blob/21f5fab97682d9bc0ae8e3b58f679fdb952f1c24/src/Simulation/TargetDefinitions/Decompositions/ExpUtil.qs#L6-L26

In [67]:
open Microsoft.Quantum.Intrinsic;

operation ExpZUtil(theta : Double, qubits : Qubit[], rotation : ((Qubit) => Unit is Adj + Ctl)) : Unit is Ctl {
    if (Length(qubits) == 1) {
        rotation(qubits[0]);
    }
    else { // Length(paulis) > 1
        within {
            SpreadZ(qubits[0], qubits[ 1 .. Length(qubits) - 1]);
        }
        apply {
            rotation(qubits[0]);
        }
    }
}

ExpZUtil

In [69]:
operation MyExpZ(theta : Double, qubits : Qubit[]) : Unit {
    ExpZUtil(theta , qubits, Rz(-2.0 * theta, _));
}

MyExpZ

In [70]:
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Measurement;

operation ApplyMyExp() : Result[] {
    use qubits = Qubit[5];
    let theta = -3.4;
    MyExpZ(theta, qubits);
    return MultiM(qubits);
}

ApplyMyExp

In [76]:
%simulate ApplyMyExp

Zero, Zero, Zero, Zero, Zero

In [6]:
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;

operation MyExpZ(theta : Double, qubits : Qubit[]) : Unit is Adj + Ctl {
    body (...) {
            MyExpZUtil(theta , qubits, Rz(-2.0 * theta, _));
        }
    adjoint(...) {
        MyExpZ(-theta, qubits);
    }
}

internal operation MyExpZUtil(
    theta : Double, qubits : Qubit[], rotation : ((Qubit) => Unit is Adj + Ctl)
) : Unit is Ctl {
    within {
        SpreadZ(qubits[0], qubits[ 1 .. Length(qubits) - 1]);
    }
    apply {
        rotation(qubits[0]);
    }
}

internal operation SpreadZ(from : Qubit, to : Qubit[]) : Unit is Adj {
    if (Length(to) > 0) {
        CNOT(to[0], from);
        if (Length(to) > 1) {
            let half = Length(to) / 2;
            SpreadZ(to[0], to[half + 1 .. Length(to) - 1]);
            SpreadZ(from, to[1 .. half]);
        }
    }
}

@EntryPoint()
operation TestMyExpZ() : Result {
    let numQubits = 4;
    use qubits = Qubit[numQubits];
    let pauliArr = ConstantArray<Pauli>(numQubits, PauliZ);
    let theta = PI() / 2.0;
    let (idx0, idx1) = (0, numQubits - 1);
    if idx0 == idx1 { fail "The indices must be different"; }
    within {
        H(qubits[idx0]);
    }
    apply {
        within {
            CNOT(qubits[idx0], qubits[idx1]);
        }
        apply {
            MyExpZ(theta, qubits);
        }
    }
    let res = M(qubits[idx0]);
    // let expected = Zero;
    // AssertQubit(expected, qubits[idx0]);
    ResetAll(qubits);
    return res;
}

MyExpZ, MyExpZUtil, SpreadZ, TestMyExpZ

In [ ]:
%